In [ ]:
# Notebook from https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7
!pip install -q scipy langchain transformers playwright html2text sentence_transformers faiss-gpu
!pip install -q --upgrade git+https://github.com/huggingface/transformers

In [ ]:
!playwright install > /dev/null
!playwright install-deps > /dev/null

In [ ]:
!pip install -U tokenizers

In [ ]:
import os
import torch
from transformers import (
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  pipeline
)

from transformers import BitsAndBytesConfig




from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain


#################################################################
# Tokenizer
#################################################################

model_name="mistralai/Mistral-7B-Instruct-v0.1"
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir="."
)
#load_in_4bits=True)

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
article_a_indexer="https://www.droits-salaries.com/542107800-otis/54210780003943-siege/T09221025128-accord-collectif-relatif-au-teletravail-otis-france-teletravail.shtml"
SITE="DROITS"
#SITE="MAITRE"

In [ ]:
import nest_asyncio

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

nest_asyncio.apply()

# Articles to index
articles = [article_a_indexer]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

# Converts HTML to plain text 
html2text = Html2TextTransformer()

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(docs[0].page_content, 'html.parser')
var_class=""
if "DROITS" in SITE:
    var_class="texte_accord"
elif "MAITRE" in SITE:
    var_class="texte-accord"
new_docs=soup.find(class_=var_class).text

In [ ]:
with open("test.txt","w") as file:
    file.write(new_docs.encode('ascii', 'ignore').decode('ascii'))

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./test.txt")
docs=loader.load()

In [ ]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=1000, 
                                      chunk_overlap=100)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

In [ ]:
prompt_template = """

### [ROLE] Role: Vous êtes un expert juridique des accords d'entreprise sans compétence de programmation informatique, seulement la lecture et la compréhension des textes. Tu ne connais aucun langage de programmation, ni les expressions régulières.


### [INST] Instruction: Votre travail consiste à peupler et mettre à jour une base des données nécessaires pour réaliser des études économétriques poussées sur le télétravail.
        Pour cela, vous devez prendre les textes des accords d'entreprise dans lesquelles figurent les informations à jour que l'on vous fourni, et d'extraire précisément l'information spécifique dont vous avez besoin.
        Vous ne devez extraire que le nombre, et uniquement le nombre. Le nombre peut être écrit en lettres ou en chiffres. (exemple : deux ou 2 )
        Si vous ne trouver pas la réponse dans le texte vous devez le dire et ne pas chercher à en fournir un autre nombre.
        Avant de répondre, vérifier que la réponse se trouve bien dans le texte indiqué.
        Impératif pour votre raisonnement : Déconnecter les liens entre les périodicités (semaine, mois, trimestre, année)! Faire comme si les semaines, les mois, les trimestres et les années sont des concepts indépendants : dans un mois, il y a donc 0 semaine ; dans un trimestre, 0 mois et 0 semaine; dans une année, 0 trimestre, 0 mois et 0 semaine !
{context}

### QUESTION:
{question} [/INST]
"""

In [ ]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dict_reponse=dict()

In [ ]:
def guess_reponse_booleenne(reponse):
    reponse_booleenne=None
    if reponse["text"].startswith("\nOui") or reponse["text"].startswith("Oui") :
        reponse_booleenne=1
    elif reponse["text"].startswith("\nNon") or reponse["text"].startswith("Non") :
        reponse_booleenne=0
    return reponse_booleenne

In [ ]:
def guess_reponse_durée(reponse):
    reponse_booleenne=None
    if "indéterminée" in reponse["text"].lower() :
        reponse_booleenne=0
    elif "déterminée" in reponse["text"].lower():
        reponse_booleenne=1
    return reponse_booleenne

In [ ]:
import re
def guess_reponse_nombre(reponse):
    reponse_nombre=None
    REGEX=r".*nombre=(\d+)"
    numbers=re.findall(REGEX,reponse["text"])
    if numbers:
        reponse_nombre=numbers[0]
    return reponse_nombre

In [ ]:
Q_DUREE="S'il est fait mention d'une durée de l'accord, est-ce que l'accord est à durée déterminée ou à durée indéterminée ? déterminée=1 ou indéterminée=0 ou NA?"
Q_REVERS="S'il est fait mention d'une clause et d'un article de réversibilité, est-ce que l'accord a une clause ou un article de réversibilité ? Oui=1 ou non=0 ou NA?"
Q_ADAPT="S'il est fait mention d'une période d'adaptation, est-ce que l'accord comprend une période d'adaptation ? Oui=1 ou non=0 ou NA?"
#Q_TTREG="S'il est fait mention d'un télétravail régulier, est-ce que l'accord détaille le télétravail régulier ? Oui=1 ou non=0 ou NA?"
#Q_TTOCA="S'il est fait mention d'un télétravail occasionnel, est-ce que l'accord mentionne le télétravail occasionnel ? Oui=1 ou non=0 ou NA?"
#Q_TTEXC="S'il est fait mention d'un télétravail exceptionnel, est-ce que l'accord mentionne le télétravail exceptionnel ? Oui=1 ou non=0 ou NA?"
Q_TTREG="Est-ce qu'au moins un paragraphe mentionne du télétravail régulier ?"
Q_TTOCA="Est-ce qu'au moins un paragraphe mentionne du télétravail occasionnel ?"
Q_TTEXC="Est-ce qu'au moins un paragraphe mentionne du télétravail exceptionnel ?"

In [ ]:
Q_DISPOSPERQTHENCEINTE="Est-ce qu'au moins un paragraphe mentionne des dispositifs spéciaux pour les travailleurs en situation de handicap, les femmes enceintes, les séniors ou les personnes présentant des fragilités ou des problèmes de santé ?"
reponse= rag_chain.invoke(Q_DISPOSPERQTHENCEINTE)
print("Q_DISPOSPERQTHENCEINTE:",reponse["text"])
dict_reponse["DISPOSPERQTHENCEINTE"]=guess_reponse_booleenne(reponse)

In [ ]:
Q_MONTANT_CAVIARDE="Est-ce qu'au moins un paragraphe mentionne un nombre sans le donner exactement ?"
reponse= rag_chain.invoke(Q_MONTANT_CAVIARDE)
print("Q_MONTANT_CAVIARDE:",reponse["text"])
dict_reponse["MONTANT_CAVIARDE"]=guess_reponse_booleenne(reponse)

In [ ]:
Q_TELETRAVAIL_FLEX_SANS_LIMITE="Est-ce qu'au moins un paragraphe suggère une liberté totale dans le choix des jours de télétravail, hormis quelques contraintes de présence ?"
reponse= rag_chain.invoke(Q_TELETRAVAIL_FLEX_SANS_LIMITE)
print("Q_TELETRAVAIL_FLEX_SANS_LIMITE:",reponse["text"])
dict_reponse["TELETRAVAIL_FLEX_SANS_LIMITE"]=guess_reponse_booleenne(reponse)

In [ ]:
Q_TT_REG_NOMBRE_FORMULE=Q_TTREG+"Si oui, combien de formules (exemples de formule : si temps plein et temps partiel = 2, si choix entre 1 à 3 jours de télétravail régulier = 3) sont proposées pour du télétravail régulier ? Retourne à la fin 'nombre=max(tes réponses)'"
reponse= rag_chain.invoke(Q_TT_REG_NOMBRE_FORMULE)
print("Q_TT_REG_NOMBRE_FORMULE:",reponse["text"])
dict_reponse["TT_REG_NOMBRE_FORMULE"]=guess_reponse_nombre(reponse)

In [ ]:
reponse= rag_chain.invoke(Q_DUREE)
print("Q_DUREE:",reponse["text"])
dict_reponse["DUREE"]=guess_reponse_durée(reponse)
reponse= rag_chain.invoke(Q_REVERS)
print("Q_REVERS:",reponse["text"])
dict_reponse["REVERS"]=guess_reponse_booleenne(reponse)
reponse= rag_chain.invoke(Q_ADAPT)
print("Q_ADAPT:",reponse["text"])
dict_reponse["ADAPT"]=guess_reponse_booleenne(reponse)
reponse= rag_chain.invoke(Q_TTREG)
print("Q_TTREG:",reponse["text"])
dict_reponse["TTREG"]=guess_reponse_booleenne(reponse)
reponse= rag_chain.invoke(Q_TTOCA)
print("Q_TTOCA:",reponse["text"])
dict_reponse["TTOCA"]=guess_reponse_booleenne(reponse)
reponse= rag_chain.invoke(Q_TTEXC)
print("Q_TTEXC:",reponse["text"])
dict_reponse["TTEXC"]=guess_reponse_booleenne(reponse)

In [ ]:
dict_reponse

In [ ]:
CONSIGNE_REFUS="Sinon répond 0."

In [ ]:
PERIODICITE={"MOIS":"mois","TRIM":"trimestre","ANNEE":"année", "SEM":"semaine"}

In [ ]:
for (k,v) in PERIODICITE.items():
    Q_TT=f"Hors jours supplémentaires ou exceptionnels, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail maximum autorisé explicitement exprimé par {v} ?"
    reponse= rag_chain.invoke(Q_TT)
    text_reponse=reponse["text"]
    print(f"Q_TT{k}:",reponse["text"])
    dict_reponse[f"TT{k}"]=guess_reponse_booleenne(reponse)
    #Q_TT_NOMBRE=f"Extrait le nombre de jour de télétravail maximum autorisé explicitement exprimé par {v} seulement à partir du texte suivant : {text_reponse}"
    Q_TT_NOMBRE=Q_TT+f"Si oui, quel est le nombre de jour de télétravail supplémentaires explicitement exprimé et nécessairement et seulement exprimé par {v} pour des dispositions spéciales. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
    reponse= rag_chain.invoke(Q_TT_NOMBRE)
    print(f"Q_TT{k}_NOMBRE:",reponse["text"])
    dict_reponse[f"TT{k}_NOMBRE"]=guess_reponse_nombre(reponse)

In [ ]:
for (k,v) in PERIODICITE.items():
    Q_TTDISPOSPE=f"Hors jours supplémentaires classiques ou exceptionnels, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail supplémentaires explicitement exprimé et nécessairement et seulement exprimé par {v} pour des dispositions spéciales ?"
    reponse= rag_chain.invoke(Q_TTDISPOSPE)
    print(f"Q_TTDISPOSPE{k}:",reponse["text"])
    dict_reponse[f"TTDISPOSPE{k}"]=guess_reponse_booleenne(reponse)
    Q_TTDISPOSPE_NOMBRE=Q_TTDISPOSPE+f"Si oui, quel est le nombre de jour de télétravail supplémentaires explicitement exprimé et nécessairement et seulement exprimé par {v} pour des dispositions spéciales. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
    reponse= rag_chain.invoke(Q_TTDISPOSPE_NOMBRE)
    print(f"Q_TTDISPOSPE{k}_NOMBRE:",reponse["text"])
    dict_reponse[f"TTDISPOSPE{k}_NOMBRE"]=guess_reponse_nombre(reponse)

In [ ]:
for (k,v) in PERIODICITE.items():
    Q_TTSUP=f"Hors dispositions particulières ou spéciales, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail supplémentaires explicitement exprimé et nécessairement et seulement exprimé par {v} ?"
    reponse= rag_chain.invoke(Q_TTSUP)
    print(f"Q_TTSUP{k}:",reponse["text"])
    dict_reponse[f"TTSUP{k}"]=guess_reponse_booleenne(reponse)
    Q_TTSUP_NOMBRE=Q_TTSUP+f"Si oui, quel est le nombre de jour de télétravail supplémentaires explicitement exprimé et nécessairement et seulement exprimé par {v}, hors dispositions spéciales ? Si oui, retourne à la fin 'nombre=max(tes réponses)'" 
    reponse= rag_chain.invoke(Q_TTSUP_NOMBRE)
    print(f"Q_TTSUP{k}_NOMBRE:",reponse["text"])
    dict_reponse[f"TTSUP{k}_NOMBRE"]=guess_reponse_nombre(reponse)

In [ ]:
for (k,v) in PERIODICITE.items():
    Q_PRESJOUR=f"Est-ce qu'au moins un paragraphe mentionne un nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v} ?"
    reponse= rag_chain.invoke(Q_PRESJOUR)
    print(f"Q_PRESJOUR{k}:",reponse["text"])
    dict_reponse[f"TTPRESJOUR{k}"]=guess_reponse_booleenne(reponse)
    Q_PRESJOUR_NOMBRE=Q_PRESJOUR+f"Si oui, sélectionne seulement les paragraphes mentionnant un nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v} et s'il existe donne le nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v}. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
    reponse= rag_chain.invoke(Q_PRESJOUR_NOMBRE)
    print(f"Q_PRESJOUR{k}_NOMBRE:",reponse["text"])
    dict_reponse[f"PRESJOUR{k}_NOMBRE"]=guess_reponse_nombre(reponse)

In [ ]:
PERIODICITE2={"JOUR":"jour","MOIS":"mois","TRIM":"trimestre","ANNEE":"année", "SEM":"semaine"}

In [ ]:
for (k,v) in PERIODICITE2.items():
    Q_COMP=f"Hors indemnité d'équipement, est-ce qu'au moins un paragraphe mentionne une indemnité forfaitaire explicitement exprimée par {v} ?"
    reponse= rag_chain.invoke(Q_COMP)
    print(f"Q_COMP{k}:",reponse["text"])
    dict_reponse[f"COMP{k}"]=guess_reponse_booleenne(reponse)
    Q_COMP_NOMBRE=Q_COMP+f"Si oui, sélectionne en interne seulement les paragraphes mentionnant une indemnité forfaitaire explicitement exprimé par {v}, puis s'il existe donne le montant explicitement exprimé par {v}. Si oui, retourne à la fin 'nombre=max(ta réponse)'"
    reponse= rag_chain.invoke(Q_COMP_NOMBRE)
    print(f"Q_COMP{k}_NOMBRE:",reponse["text"])
    dict_reponse[f"COMP{k}_NOMBRE"]=guess_reponse_nombre(reponse)
    if "MOIS" in k:
        Q_COMP_NOMBRE_BASE=Q_COMP+f"Si oui, sélectionne en interne seulement les paragraphes mentionnant une indemnité forfaitaire explicitement exprimé par {v}, puis s'il existe plusieurs montants explicitement exprimé par {v}, par nombre de jour. Si oui, retourne à la fin 'nombre=base_mensuelle_pour_un_jour_de_teletravail(ta réponse)'"
        reponse= rag_chain.invoke(Q_COMP_NOMBRE_BASE)
        print(f"Q_COMP{k}_BASE_NOMBRE:",reponse["text"])
        dict_reponse[f"COMP{k}_BASE_NOMBRE"]=guess_reponse_nombre(reponse)

In [ ]:
Q_COMPE="Est-ce qu'au moins un paragraphe mentionne une indemnité spécialement pour l'achat d'équipements ?"
reponse= rag_chain.invoke(Q_COMPE)
print("Q_COMPE:",reponse["text"])
dict_reponse["COMPE"]=guess_reponse_booleenne(reponse)
Q_COMPE_NOMBRE=Q_COMPE+"Si oui, sélectionne seulement les paragraphes mentionnant une indemnité spécialement pour l'achat d'équipements et s'il existe donne le montant lié de l'indemnité d'équipement. Si oui, retourne à la fin 'nombre=max(ta réponse)'"
reponse= rag_chain.invoke(Q_COMPE_NOMBRE)
print("Q_COMPE_NOMBRE:",reponse["text"])
dict_reponse["COMPE_NOMBRE"]=guess_reponse_nombre(reponse)

In [ ]:
dict_reponse